In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np

%matplotlib inline

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def parse_from_txt(file):
    '''This takes the aligned, merged file from terminal analysis. We first get rid of any genes that do not
    start with ATG and end with TAA. Then we append the resulting genes to a list.'''
    gene = ""
    text_file = open(file, "r") #open and read the text file containing aligned library reads
    lines = text_file.readlines()
    text_file.close()
    
    record = [] #this is list of genes that will be counted later
    for x in lines: #separate out each read to be analyzed individually
        '''The start codon follow a tab, and the stop codon should preceed a tab'''
        for i in range(100): #scan the first 100 letters for \tATG
            if x[i:i+4] == "CATG":
                if x[i+391:i+394] == "TAA": #391 letters after the tab, there should be a stop codon and a tab
                        gene = x[i+1:i+394] #only take records that agree with both of these conditions
                        record.append(gene) #append to a list for further analysis
    return record

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def parse_from_txt_mini(file):
    '''This takes the aligned, merged file from terminal analysis. We first get rid of any genes that do not
    start with ATG and end with TAA. Then we append the resulting genes to a list.'''
    gene = ""
    text_file = open(file, "r") #open and read the text file containing aligned library reads
    lines = text_file.readlines()
    text_file.close()
    
    record = [] #this is list of genes that will be counted later
    for x in lines: #separate out each read to be analyzed individually
        '''The start codon follow a tab, and the stop codon should preceed a tab'''
        for i in range(100): #scan the first 100 letters for \tATG
            if x[i:i+4] == "CATG":
                if x[i+391:i+394] == "TAA": #391 letters after the tab, there should be a stop codon and a tab
                    if x[i+112:i+115] == "CCG": #If there is the S37P mutation
                        gene = x[i+1:i+394] #only take records that agree with both of these conditions
                        record.append(gene) #append to a list for further analysis
    return record

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def count_heatmap(dna_lst):
    '''This function takes the output of parse_from_txt and counts codons. Several conditions are in place:
    First, we determine how many mutations there are per gene. We discard wildtype reads (or those with zero 
    mutations. Anything with one mutation is counted. For reads with two or more mutations, any 1-bp mutations
    are discarded, while 2+ bp mutations are counted'''

    ms2_gene = ('ATGGCTTCTAACTTTACTCAGTTCGTTCTCGTCGACAATGGCGGAAC'+
    'TGGCGACGTGACTGTCGCCCCAAGCAACTTCGCTAACGGGGTCGCTGAATGGATCAGC'+
    'TCTAACTCGCGTTCACAGGCTTACAAAGTAACCTGTAGCGTTCGTCAGAGCTCTGCGC'+
    'AGAATCGCAAATACACCATCAAAGTCGAGGTGCCTAAAGTGGCAACCCAGACTGTTGG'+
    'TGGTGTAGAGCTTCCTGTAGCCGCATGGCGTTCGTACTTAAATATGGAACTAACCATT'+
    'CCAATTTTCGCTACGAATTCCGACTGCGAGCTTATTGTTAAGGCGATGCAAGGACTCC'+
    'TAAAAGATGGAAACCCGATTCCCTCAGCAATCGCAGCAAACTCCGGCATCTACTAA')
    
    intensities = np.zeros((131, 64))

    codon_to_num = {'TTT': 0, 'TTG': 1, 'TTA': 2, 'TTC': 3, 
                     'CTT': 4, 'CTG': 5, 'CTA': 6, 'CTC': 7,
                     'ATT': 8, 'ATG': 9, 'ATA': 10, 'ATC': 11,
                     'GTT': 12, 'GTG': 13, 'GTA': 14, 'GTC': 15,
                     'TAT': 16, 'TAG': 17, 'TAA': 18, 'TAC': 19,
                     'CAT': 20, 'CAG': 21, 'CAA': 22, 'CAC': 23,
                     'AAT': 24, 'AAG': 25, 'AAA': 26, 'AAC': 27,
                     'GAT': 28, 'GAG': 29, 'GAA': 30, 'GAC': 31,
                     'TCT': 32, 'TCG': 33, 'TCA': 34, 'TCC': 35,
                     'CCT': 36, 'CCG': 37, 'CCA': 38, 'CCC': 39,
                     'ACT': 40, 'ACG': 41, 'ACA': 42, 'ACC': 43,
                     'GCT': 44, 'GCG': 45, 'GCA': 46, 'GCC': 47,
                     'TGT': 48, 'TGG': 49, 'TGA': 50, 'TGC': 51,
                     'CGT': 52, 'CGG': 53, 'CGA': 54, 'CGC': 55,
                     'AGT': 56, 'AGG': 57, 'AGA': 58, 'AGC': 59,
                     'GGT': 60, 'GGG': 61, 'GGA': 62, 'GGC' :63} 
    
    '''Each amino acid has a number associated with it, which is also its position in AAs.
    i.e. A = 0; S = 1; etc. AA_code allows us to move back and forth between codon and AA #.
    Lists checked on 6/27'''

    AA_code = {44: 0, 45: 0, 46: 0, 47: 0, 32: 1, 33: 1, 34: 1, 35: 1, 56: 1, 59: 1, 
                 40: 2, 41: 2, 42: 2, 43: 2, 12: 3, 13: 3, 14: 3, 15: 3, 48: 4, 51: 4, 
                 29: 5, 30: 5, 28: 6, 31: 6, 25: 7, 26: 7, 52: 8, 53: 8, 54: 8, 55: 8, 
                 57: 8, 58: 8, 21: 9, 22: 9, 24: 10, 27: 10, 9: 11, 8: 12, 10: 12, 11: 12, 
                 1: 13, 2: 13, 4: 13, 5: 13, 6: 13, 7: 13, 20: 14, 23: 14, 0: 15, 3: 15, 
                 16: 16, 19: 16, 49: 17, 60: 18, 61: 18, 62: 18, 63: 18, 36: 19, 37: 19, 
                 38: 19, 39: 19, 17: 20, 18: 20, 50: 20}
    
    AAs = ['A', 'S', 'T', 'V', 'C', 'E', 'D', 'K', 'R', 'Q', 'N', 'M', 'I', 'L', 
           'H', 'F', 'Y', 'W', 'G', 'P', '*']
    
    for dna_read in dna_lst: #dna_read counts where you are in sequencing reads
        '''first we count how many mutations there are in a read'''
        number_of_mutations = 0 #number_of_mutations counts number of mutations in a dna_read (seq reads)
        for codon_number in range(131): #codon_number is the residue number in MS2 CP
            dna_base_num = codon_number * 3 #dna_base_num is dna base pair number
            wildtype = ms2_gene[dna_base_num:dna_base_num+3]
            experiment_codon = dna_read[dna_base_num:dna_base_num+3]
            if wildtype != experiment_codon:
                #counter increases if the codon is not wildtype
                number_of_mutations += 1
        if number_of_mutations > 1:
            '''if number_of_mutations > 1, get rid of any mutations that are 1 bp different from wildtype'''
            for second_loop_codon_number in range(131): #loop through read again
                dna_base_num = second_loop_codon_number * 3 #get DNA base pair number
                wildtype = ms2_gene[dna_base_num:dna_base_num+3] #slice out wildtype codon
                experiment_codon = dna_read[dna_base_num:dna_base_num+3] #slice out experiment codon
                read_level_mutation_counter = 0 #counter for the number of basepairs that are different from wildtype
                if wildtype != experiment_codon: #if the two codons aren't the same, count # of differences
                    for base in range(3):
                        if experiment_codon[base] != wildtype[base]:
                            read_level_mutation_counter += 1
                    if read_level_mutation_counter == 1:
                        pass #do not count the codon if it is a 1-bp mutation
                    elif experiment_codon in codon_to_num: #otherwise, add it to the intesities array
                        intensities[second_loop_codon_number, codon_to_num[experiment_codon]] += 1
        if number_of_mutations == 0:
            '''if wildtype, toss'''
        if number_of_mutations == 1:
            '''keep the read if there is one total mutation'''
            for third_loop_codon_number in range(131):
                dna_base_num = third_loop_codon_number * 3
                wildtype = ms2_gene[dna_base_num:dna_base_num+3]
                experiment_codon = dna_read[dna_base_num:dna_base_num+3]
                if wildtype != experiment_codon: #find the mutation in the read
                    intensities[third_loop_codon_number, codon_to_num[experiment_codon]] += 1
    return intensities

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def count_heatmap_mini(dna_lst):
    '''This function takes the output of parse_from_txt and counts codons. Several conditions are in place:
    First, we determine how many mutations there are per gene. We discard wildtype reads (or those with zero 
    mutations. Anything with one mutation is counted. For reads with two or more mutations, any 1-bp mutations
    are discarded, while 2+ bp mutations are counted'''

    ms2_gene = ('ATGGCTTCTAACTTTACTCAGTTCGTTCTCGTCGACAATGGCGGAAC'+
    'TGGCGACGTGACTGTCGCCCCAAGCAACTTCGCTAACGGGGTCGCTGAATGGATCAGC'+
    'TCTAACCCGCGTTCACAGGCTTACAAAGTAACCTGTAGCGTTCGTCAGAGCTCTGCGC'+
    'AGAATCGCAAATACACCATCAAAGTCGAGGTGCCTAAAGTGGCAACCCAGACTGTTGG'+
    'TGGTGTAGAGCTTCCTGTAGCCGCATGGCGTTCGTACTTAAATATGGAACTAACCATT'+
    'CCAATTTTCGCTACGAATTCCGACTGCGAGCTTATTGTTAAGGCGATGCAAGGACTCC'+
    'TAAAAGATGGAAACCCGATTCCCTCAGCAATCGCAGCAAACTCCGGCATCTACTAA')
    
    intensities = np.zeros((131, 64))

    codon_to_num = {'TTT': 0, 'TTG': 1, 'TTA': 2, 'TTC': 3, 
                     'CTT': 4, 'CTG': 5, 'CTA': 6, 'CTC': 7,
                     'ATT': 8, 'ATG': 9, 'ATA': 10, 'ATC': 11,
                     'GTT': 12, 'GTG': 13, 'GTA': 14, 'GTC': 15,
                     'TAT': 16, 'TAG': 17, 'TAA': 18, 'TAC': 19,
                     'CAT': 20, 'CAG': 21, 'CAA': 22, 'CAC': 23,
                     'AAT': 24, 'AAG': 25, 'AAA': 26, 'AAC': 27,
                     'GAT': 28, 'GAG': 29, 'GAA': 30, 'GAC': 31,
                     'TCT': 32, 'TCG': 33, 'TCA': 34, 'TCC': 35,
                     'CCT': 36, 'CCG': 37, 'CCA': 38, 'CCC': 39,
                     'ACT': 40, 'ACG': 41, 'ACA': 42, 'ACC': 43,
                     'GCT': 44, 'GCG': 45, 'GCA': 46, 'GCC': 47,
                     'TGT': 48, 'TGG': 49, 'TGA': 50, 'TGC': 51,
                     'CGT': 52, 'CGG': 53, 'CGA': 54, 'CGC': 55,
                     'AGT': 56, 'AGG': 57, 'AGA': 58, 'AGC': 59,
                     'GGT': 60, 'GGG': 61, 'GGA': 62, 'GGC' :63} 
    
    '''Each amino acid has a number associated with it, which is also its position in AAs.
    i.e. A = 0; S = 1; etc. AA_code allows us to move back and forth between codon and AA #.
    Lists checked on 6/27'''

    AA_code = {44: 0, 45: 0, 46: 0, 47: 0, 32: 1, 33: 1, 34: 1, 35: 1, 56: 1, 59: 1, 
                 40: 2, 41: 2, 42: 2, 43: 2, 12: 3, 13: 3, 14: 3, 15: 3, 48: 4, 51: 4, 
                 29: 5, 30: 5, 28: 6, 31: 6, 25: 7, 26: 7, 52: 8, 53: 8, 54: 8, 55: 8, 
                 57: 8, 58: 8, 21: 9, 22: 9, 24: 10, 27: 10, 9: 11, 8: 12, 10: 12, 11: 12, 
                 1: 13, 2: 13, 4: 13, 5: 13, 6: 13, 7: 13, 20: 14, 23: 14, 0: 15, 3: 15, 
                 16: 16, 19: 16, 49: 17, 60: 18, 61: 18, 62: 18, 63: 18, 36: 19, 37: 19, 
                 38: 19, 39: 19, 17: 20, 18: 20, 50: 20}
    
    AAs = ['A', 'S', 'T', 'V', 'C', 'E', 'D', 'K', 'R', 'Q', 'N', 'M', 'I', 'L', 
           'H', 'F', 'Y', 'W', 'G', 'P', '*']
    
    for dna_read in dna_lst: #dna_read counts where you are in sequencing reads
        '''first we count how many mutations there are in a read'''
        number_of_mutations = 0 #number_of_mutations counts number of mutations in a dna_read (seq reads)
        for codon_number in range(131): #codon_number is the residue number in MS2 CP
            dna_base_num = codon_number * 3 #dna_base_num is dna base pair number
            wildtype = ms2_gene[dna_base_num:dna_base_num+3]
            experiment_codon = dna_read[dna_base_num:dna_base_num+3]
            if wildtype != experiment_codon:
                #counter increases if the codon is not wildtype
                number_of_mutations += 1
        if number_of_mutations > 1:
            '''if number_of_mutations > 1, get rid of any mutations that are 1 bp different from wildtype'''
            for second_loop_codon_number in range(131): #loop through read again
                dna_base_num = second_loop_codon_number * 3 #get DNA base pair number
                wildtype = ms2_gene[dna_base_num:dna_base_num+3] #slice out wildtype codon
                experiment_codon = dna_read[dna_base_num:dna_base_num+3] #slice out experiment codon
                read_level_mutation_counter = 0 #counter for the number of basepairs that are different from wildtype
                if wildtype != experiment_codon: #if the two codons aren't the same, count # of differences
                    for base in range(3):
                        if experiment_codon[base] != wildtype[base]:
                            read_level_mutation_counter += 1
                    if read_level_mutation_counter == 1:
                        pass #do not count the codon if it is a 1-bp mutation
                    elif experiment_codon in codon_to_num: #otherwise, add it to the intesities array
                        intensities[second_loop_codon_number, codon_to_num[experiment_codon]] += 1
        if number_of_mutations == 0:
            '''if wildtype, toss'''
        if number_of_mutations == 1:
            '''keep the read if there is one total mutation'''
            for third_loop_codon_number in range(131):
                dna_base_num = third_loop_codon_number * 3
                wildtype = ms2_gene[dna_base_num:dna_base_num+3]
                experiment_codon = dna_read[dna_base_num:dna_base_num+3]
                if wildtype != experiment_codon: #find the mutation in the read
                    intensities[third_loop_codon_number, codon_to_num[experiment_codon]] += 1
    return intensities

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def translate_NNK_array(nparray1):
    '''Takes the codon-counted np array and turns it into a amino acid np array, discarding 
    anything that ends in C or A and is thus not from an NNK library'''
    
    AA_code = {44: 0, 45: 0, 46: 0, 47: 0, 32: 1, 33: 1, 34: 1, 35: 1, 56: 1, 59: 1, 
                 40: 2, 41: 2, 42: 2, 43: 2, 12: 3, 13: 3, 14: 3, 15: 3, 48: 4, 51: 4, 
                 29: 5, 30: 5, 28: 6, 31: 6, 25: 7, 26: 7, 52: 8, 53: 8, 54: 8, 55: 8, 
                 57: 8, 58: 8, 21: 9, 22: 9, 24: 10, 27: 10, 9: 11, 8: 12, 10: 12, 11: 12, 
                 1: 13, 2: 13, 4: 13, 5: 13, 6: 13, 7: 13, 20: 14, 23: 14, 0: 15, 3: 15, 
                 16: 16, 19: 16, 49: 17, 60: 18, 61: 18, 62: 18, 63: 18, 36: 19, 37: 19, 
                 38: 19, 39: 19, 17: 20, 18: 20, 50: 20}
    
    '''Rearranged so output is in a logical order'''
    
    AAs = ['A', 'S', 'T', 'V', 'C', 'E', 'D', 'K', 'R', 'Q', 'N', 'M', 'I', 'L', 
           'H', 'F', 'Y', 'W', 'G', 'P', '*']

    
    NNK_only = [0, 1, 4, 5, 8, 9, 12, 13, 16, 17, 20, 21,
                    24, 25, 28, 29, 32, 33, 36, 37, 40, 41,
                    44, 45, 48, 49, 52, 53, 56, 57, 60, 61]
    
    array_translate = np.zeros((131, 21))
    
    for i in NNK_only: #Loop through NNK_only (which is codon numbers excluding those ending in C or A)
        x = AA_code[i] #Get the amino acid location where the codon row should be added
        array_translate[:,x] += nparray1[:,i] #Add the entire row to the appropriate amino acid row

    return array_translate

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def heatmap(array):
    '''Combines all analyses into one function that outputs the counted array'''
    array_p = parse_from_txt(array)
    array_c = count_heatmap(array_p)
    array_trans = translate_NNK_array(array_c)
    return array_trans

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def heatmap_mini(array):
    '''Combines all analyses into one function that outputs the counted array'''
    array_p = parse_from_txt_mini(array)
    array_c = count_heatmap_mini(array_p)
    array_trans = translate_NNK_array(array_c)
    return array_trans

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def chunk_heatmap(array):
    '''Divides heatmap into five chunks (which is how they were cloned)
    We compare percent abundance within each chunk between starting (plasmid) library
    and final (VLP) library. This comparison becomes our Apparent Fitness Score'''
    cutoffs = [0, 27, 53, 79, 105, 130]
    sums = []
    chunked = np.zeros((131,21))
    for chunk in range(5):
        summed_chunk = sum(sum(array[cutoffs[chunk]:cutoffs[chunk+1],:])) #grab the array from 0:27, 27:53, etc. Sum all points.
        sums.append(summed_chunk) #Append that sum to a list
    for chunk in range(5):
        for codon in range(131):
            for amino_acid in range(21):
                if codon in range(cutoffs[chunk],cutoffs[chunk+1]):
                    chunked[codon,amino_acid] = array[codon,amino_acid] / sums[chunk]
    return chunked

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def heatmap_diff(array1, array2):
    '''Compares two heatmaps. Array 1 / Array 2, with zeros in array2 masked with np.nans'''
    diff = np.zeros((131,21))
    for codon in range(131):
        for amino_acid in range(21):
            if array2[codon,amino_acid] == 0: #if there's a zero in array2, the position becomes nan
                diff[codon,amino_acid] = np.nan
            else:
                diff[codon,amino_acid] = array1[codon,amino_acid] / array2[codon,amino_acid] #else, divide, add to diff
                
    return diff

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def heatmap_avgs(array1, array2, array3):
    '''Takes average of three heatmaps'''
    log = np.zeros((131,21))
    m = np.array([array1, array2, array3]) #this array of arrays means we can use axis=0 to get the mean
    mean = np.nanmean(m, axis=0) #nanmean masks ignores nan values
    for i in range(131):
        for n in range(21):
            mean[i,n] = max(mean[i,n], .0001) #if the mean is zero, replace with .0001 so the resulting log value is -4
            log[i,n] = np.log10(mean[i,n]) #take the log of the mean value
    return log #this is the finalized heatmap with apparent fitness scores

In [ ]:
'''Data processing for Apparent Fitness Landscape'''

def plot_heatmap(heatmap_np, name):
    heatmap_np = heatmap_np[1:130,:]

    ytix = []
    for i in range(1,13):
        i = i * 10
        ytix.append(i)


    data = heatmap_np
    data = np.ma.masked_invalid(data)
    labels = 'ASTVCEDKRQNMILHFYWGP*'
    
    ms2_protein = ('MASNFTQFVLVDNGGTGDVTVAPSNFANGVAEWISPNPRSQAYKVTC'+
    'SVRQSSAQNRKYTIKVEVPKVATQTVGGVELPVAAWRSYLNMELTIPIFATNSDCELIVKA'+
    'MQGLLKDGNPIPSAIAANSGIY*')

    AAs_dict = {'A': 0, 'S': 1, 'T': 2, 'V': 3, 'C': 4, 'E': 5, 'D': 6, 'K': 7, 'R': 8, 'Q': 9, 
                'N': 10, 'M': 11, 'I': 12, 'L': 13, 'H': 14, 'F': 15, 'Y': 16, 'W': 17, 'G': 18, 'P': 19, '*': 20}

    wt_np = np.zeros((131,21))
    for i in range(len(ms2_protein)):
        wt_np[i,AAs_dict[ms2_protein[i]]] = True
    wt_np = wt_np[1:130,:]

    fig, ax = plt.subplots(figsize = (15,20))
    
    ax.patch.set(hatch='', color="silver")
    im = ax.pcolor(data, cmap='RdBu', vmax = 2, vmin = -2, edgecolor='black', linestyle=':', lw=1)
    plt.colorbar(im, aspect=40)
    plt.title('Apparent Fitness Landscape', fontsize=20)
    plt.ylabel("Residue Number", fontsize=18)
    plt.xlabel("Amino Acid", fontsize=18)

    # Shift ticks to be at 0.5, 1.5, etc
    ax.xaxis.set(ticks=np.arange(0.5, len(labels)), ticklabels=labels)
    ax.yaxis.set(ticks=np.arange(9.5, 129, 10), ticklabels=ytix)
    plt.tick_params(axis='y',
                    labelsize=14)
    
    mask = wt_np > 0
    for j, i in np.column_stack(np.where(mask)):
        ax.add_patch(
            mpatches.Rectangle(
                (i, j),     # (x,y)
                1,          # width
                1,          # height
                fill=False, 
#                 edgecolor='blue',
                snap=False,
                    hatch='///' # the more slashes, the denser the hash lines 
          ))
    
    plt.savefig(name, dpi=1000, format='pdf')
    plt.show()
    
    return

In [ ]:
'''Figure 2: The Apparent Fitness Landscape'''

def heatmap_diff_bioreps(array1, array2, array3, array4, array5, array6, name):
    '''All analysis to generate a heatmap plot'''
    a = heatmap(array1)
    b = heatmap(array2)
    c = heatmap(array3)
    d = heatmap(array4)
    e = heatmap(array5)
    f = heatmap(array6)
        
    ch_a = chunk_heatmap(a)
    ch_b = chunk_heatmap(b)
    ch_c = chunk_heatmap(c)
    ch_d = chunk_heatmap(d)
    ch_e = chunk_heatmap(e)
    ch_f = chunk_heatmap(f)
    
    d_a = heatmap_diff(ch_d, ch_a)
    e_b = heatmap_diff(ch_e, ch_b)
    f_c = heatmap_diff(ch_f, ch_c)
    
    final = heatmap_avgs(d_a, e_b, f_c)
    
    plot_heatmap(final, name)
        
    return final

htmp_avg = heatmap_diff_bioreps('wt-plas-r1.txt', 'wt-plas-r2.txt', 'wt-plas-r3.txt', 
                                 'wt-vlp-r1.txt', 'wt-vlp-r2.txt', 'wt-vlp-r3.txt', 'Wildtype AFL Challenge')

In [ ]:
'''Figure 2: The Apparent Fitness Landscape'''

def heatmap_diff_bioreps_mini(array1, array2, array3, array4, array5, array6, name):
    '''All analysis to generate a heatmap plot'''
    a = heatmap_mini(array1)
    b = heatmap_mini(array2)
    c = heatmap_mini(array3)
    d = heatmap_mini(array4)
    e = heatmap_mini(array5)
    f = heatmap_mini(array6)
        
    ch_a = chunk_heatmap(a)
    ch_b = chunk_heatmap(b)
    ch_c = chunk_heatmap(c)
    ch_d = chunk_heatmap(d)
    ch_e = chunk_heatmap(e)
    ch_f = chunk_heatmap(f)
    
    d_a = heatmap_diff(ch_d, ch_a)
    e_b = heatmap_diff(ch_e, ch_b)
    f_c = heatmap_diff(ch_f, ch_c)
    
    final = heatmap_avgs(d_a, e_b, f_c)
    
    plot_heatmap(final, name)
        
    return final

htmp_avg_mini = heatmap_diff_bioreps_mini('mini-plas-r1.txt', 'mini-plas-r2.txt', 'mini-plas-r3.txt', 
                                 'mini-vlp-r1.txt', 'mini-vlp-r2.txt', 'mini-vlp-r3.txt', 'S37P AFL')

In [ ]:
#write AFL to csv
def using_multiindex(A, columns):
    shape = A.shape
    index = pd.MultiIndex.from_product([range(s)for s in shape], names=columns)
    df = pd.DataFrame({'AFS':A.flatten()}, index=index).reset_index()
    return df
columns = ['Position', 'Residue']

wt_df = using_multiindex(htmp_avg, columns)
wt_df.to_csv("wt_AFL.csv")

wt_df = using_multiindex(htmp_avg_mini, columns)
wt_df.to_csv("mini_AFL.csv")

In [ ]:
def shannon_calc(array):
    '''Calculates Shannon Entropy, or a measure of diversity, on the translated heatmap'''
    summed = np.zeros((131,1))
    array_sum = np.zeros((131,1))
    
    for i in range(131):
        summed[i,0] = sum(array[i,:]) #Sum by each row (residue number)
    
    array_avg = array / summed 
    #Divide each point in the array by the average of its row to get a percent abundance per residue
    
    for i in range(131):
        for n in range(21):
            array_avg[i,n] = max(array_avg[i,n], .00001) 
            #Array_avg is the maximum between .00001 and its value (exclude zeros)

    array_log = array_avg * np.log10(array_avg) #Probability times the log of the probability
    
    for i in range(131):
        array_sum[i,0] = -sum(array_log[i,:]) #Sum up each row by residue number

    return array_sum

In [ ]:
def shannon_avg(array1, array2, array3):
    '''Takes three counted arrays and returns average shannon entropy'''
    avg = np.zeros((131,1))
    for i in range(131):
        avg[i,0] = (array1[i,0] + array2[i,0] + array3[i,0])/3
    return avg

In [ ]:
def shannon_avg_from_txt(txt1, txt2, txt3):
    
    '''From heatmap to graphs, this function performs and plots 
    Shannon entropy calculations on three biological replicates'''
        
    a = heatmap(txt1)
    b = heatmap(txt2)
    c = heatmap(txt3)
    
    sh_a = shannon_calc(a)
    sh_b = shannon_calc(b)
    sh_c = shannon_calc(c)
    
    avg = shannon_avg(sh_a, sh_b, sh_c)
    return avg

In [ ]:
def shannon_avg_from_txt_mini(txt1, txt2, txt3):
    
    '''From heatmap to graphs, this function performs and plots 
    Shannon entropy calculations on three biological replicates'''
        
    a = heatmap_mini(txt1)
    b = heatmap_mini(txt2)
    c = heatmap_mini(txt3)
    
    sh_a = shannon_calc(a)
    sh_b = shannon_calc(b)
    sh_c = shannon_calc(c)
    
    avg = shannon_avg(sh_a, sh_b, sh_c)
    return avg

In [ ]:
def shannon_diff(VLP, PLAS, name):
    '''inputs two shannon entropy arrays, returns the subtracted difference between them'''
    shan_dif = np.subtract(VLP,PLAS)
    
    return shan_dif

In [ ]:
def plot_shannon_heat_mult(array1, array2, array3, name):
    
    '''Inputs any 3 measures of shannon entropy and plots them as a heatmap'''

    fig, ax = plt.subplots(figsize = (6, 8))
    
    plt.subplot(1,3,1)
    im = plt.pcolor(array1, cmap='RdBu', vmax = .5, vmin = -1, linestyle=':', lw=1)
    fig.colorbar(im)
    plt.title('Wildtype', fontsize=14)
    plt.ylabel("Residue Number", fontsize=14)
    plt.tick_params(
                    axis='x',          # changes apply to the x-axis
                    which='both',      # both major and minor ticks are affected
                    bottom='off',      # ticks along the bottom edge are off
                    top='off',         # ticks along the top edge are off
                    labelbottom='off'
                    )
    plt.tick_params(axis='y',
                    labelsize=14)
    
    plt.subplot(1,3,2)
    im = plt.pcolor(array2, cmap='RdBu', vmax = .5, vmin = -1, linestyle=':', lw=1)
    fig.colorbar(im)
    plt.title('S37P', fontsize=14)
    plt.tick_params(
                    axis='x',          # changes apply to the x-axis
                    which='both',      # both major and minor ticks are affected
                    bottom='off',      # ticks along the bottom edge are off
                    top='off',         # ticks along the top edge are off
                    labelbottom='off'
                    )
    plt.tick_params(axis='y',
                    labelsize=14)
    
    plt.subplot(1,3,3)
    im = plt.pcolor(array3, cmap='RdBu', vmax = .5, vmin = -.5, linestyle=':', lw=1)
    fig.colorbar(im)
    plt.title('Difference', fontsize=14)
    plt.tick_params(
                    axis='x',          # changes apply to the x-axis
                    which='both',      # both major and minor ticks are affected
                    bottom='off',      # ticks along the bottom edge are off
                    top='off',         # ticks along the top edge are off
                    labelbottom='off'
                    )
    plt.tick_params(axis='y',
                    labelsize=14)
    
    
    fig.tight_layout()
    plt.savefig(name, dpi=600, format='svg')
    plt.show()
    return

PLAS = shannon_avg_from_txt('wt-plas-r1.txt', 'wt-plas-r2.txt', 'wt-plas-r3.txt')
VLP = shannon_avg_from_txt('wt-vlp-r1.txt', 'wt-vlp-r2.txt', 'wt-vlp-r3.txt')
rel_vlp_heat = shannon_diff(VLP, PLAS, 'WT_SE_Diff')

PLAS_mini = shannon_avg_from_txt_mini('mini-plas-r1.txt', 'mini-plas-r2.txt', 'mini-plas-r3.txt')
VLP_mini = shannon_avg_from_txt_mini('mini-vlp-r1.txt', 'mini-vlp-r2.txt', 'mini-vlp-r3.txt')
rel_vlp_mini = shannon_diff(VLP_mini, PLAS_mini, 'S37P_SE_Diff')

rel_vlps = shannon_diff(rel_vlp_mini, rel_vlp_heat, 'SE_diff_hm')
plot_shannon_heat_mult(rel_vlp_heat, rel_vlp_mini, rel_vlps, "WTvS37P_SE.svg")